In [1]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar
from datetime import datetime

In [2]:
N = norm.cdf


In [85]:
def read_from_db():
    conn = sqlite3.connect('../../options_database.db')
    df = pd.read_sql_query("SELECT * FROM options_table", conn)
    conn.close()
    return df

In [86]:
df = read_from_db()

In [87]:
len(df[(df.current_time == '2023-01-29 11:24:46.988931')])

610

In [6]:
def delta_BS_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1) - 1

def delta_BS_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1)

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
#     print(d1, d2)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T)* N(-d2) - S * N(-d1)

In [153]:
BS_PUT(1000, 1000, 1, 0, 0.0000001)

3.989422810946053e-05

In [7]:
def get_timedelta_in_years(timedelta):
    d = timedelta.days / (365.25)
    s = timedelta.seconds / (365.25*24*60*60)
    return d + s

In [195]:
def preprocessing(option_name, option_type):
    selected_option = (df[(df.full_name == option_name)][df.current_time > '2023-01-04'][df.type == option_type])
    selected_option['timedelta'] = selected_option.apply(lambda x: get_timedelta_in_years(datetime.fromisoformat(x['expiry']) - datetime.fromisoformat(x['current_time'])), axis=1)
    
    selected_option['mid_price'] = selected_option['mid_price'].fillna(0)
    selected_option['ask_price'] = selected_option['ask_price'].fillna(0)
    selected_option['bid_price'] = selected_option['bid_price'].fillna(0)
    selected_option['strike'] = selected_option['strike'].astype(float)
    # selected_option['mid_IV'] = selected_option['mid_IV'].fillna(0.001)
        
    filtered = selected_option.loc[(selected_option.bid_IV > 0) & (selected_option.bid_IV < 10) & 
                            (selected_option.ask_IV > 0) & (selected_option.ask_IV < 10) &
                             (selected_option.timedelta > 0)]
    
    
    # (selected_option.bid_IV < 10) & 
    #                                (selected_option.ask_IV < 10) &
    #                                (selected_option.mid_IV < 10) &
    
    # filtered = selected_option.loc[
    #                                (selected_option.timedelta > 0)]
    
    
    if option_type == 'put':
        filtered['BS_delta'] = delta_BS_put(filtered.underlying_price, filtered.strike, filtered.timedelta, 0, filtered.mid_IV)
    elif option_type == 'call':
        filtered['BS_delta'] = delta_BS_call(filtered.underlying_price, filtered.strike, filtered.timedelta, 0, filtered.mid_IV)
    
    filtered.index = np.arange(1, len(filtered)+1)
    
    return filtered



In [239]:
def backtest(df):
    
    asset = 0  
    initial_usdc = 0
    usdc = initial_usdc
    prev_asset = asset

    for i in range(len(df)):
        prev_asset = asset
        asset = filtered['BS_delta'].iloc[i]
        usdc = usdc - (asset - prev_asset) * filtered['underlying_price'].iloc[i]

    return asset*filtered['underlying_price'].iloc[-1] + usdc - initial_usdc

In [240]:
# datetime.strptime(selected_put.iloc[5]['expiry'], "%Y %m %d %I:%M%p")
# get_timedelta_in_years(datetime.fromisoformat(selected_put.iloc[5]['expiry']) - datetime.fromisoformat(selected_put.iloc[5]['current_time']))


In [274]:
filtered = preprocessing(option_name = 'ETH-27JAN23-1300', option_type = 'put')

/var/folders/8t/1dvfc69d0l3137hz8kyzv__80000gn/T/ipykernel_10556/746598763.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_option = (df[(df.full_name == option_name)][df.current_time > '2023-01-04'][df.type == option_type])
/var/folders/8t/1dvfc69d0l3137hz8kyzv__80000gn/T/ipykernel_10556/746598763.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['BS_delta'] = delta_BS_put(filtered.underlying_price, filtered.strike, filtered.timedelta, 0, filtered.mid_IV)


In [275]:
backtest(filtered[filtered.index % 10 != 0])

-127.95674369997391

In [276]:
option_price = filtered['mid_price'].iloc[0] * filtered['underlying_price'].iloc[0]

price_on_expiry = filtered['underlying_price'].iloc[-1]

PnL = max(0, filtered['strike'].iloc[-1] - filtered['underlying_price'].iloc[-1]) - filtered['mid_price'].iloc[0] * filtered['underlying_price'].iloc[0]

print(option_price, price_on_expiry, PnL)


118.34885750000001 1554.8 -118.34885750000001


In [224]:
# filtered